In [ ]:
# makes a fixed-sized embeddings in the first layer

# relies on outside variables: X_val, y_val, working_dir, n_classes, class_weights, CUSTOM_OBJECTS, ...

### Define a wrapper class for creating the classification model with embedding, fitting, etc.
It creates an useful object using a list of intermediate layers as the main argument.
Makes it easier to consistently use the chosen metrics, loss function, class_weights, embedding etc.

In [ ]:
# the workhorse
class BlackBox():
    
    def __init__(self, tag,\
                 layers, loss, batch_size, optimizer, epochs,\
                 metrics,\
                 mask_zeros=True,\
                 callbacks = None,\
                 embedd_file=None
                ):
        
        self.layers = layers
        self.loss = loss
        self.loss_name = self.loss.__name__ if self.loss != 'categorical_crossentropy' else 'categorical_crossentropy'
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.tag = tag
        self.epochs = epochs
        self.metrics = metrics
        self.mask_zeros = mask_zeros
        self.callbacks = callbacks

        if loss in ['categorical_crossentropy', cat_cross]:
            self.class_weight = class_weights
        else:
            self.class_weight = None
        
        self.model = Sequential()
        self.model.add(Embedding(input_dim=num_words,
                                 output_dim=50,
                                 input_length=padded_length,
                                 mask_zero=self.mask_zeros))
        
        for layer in layers:
            self.model.add(layer)
        self.model.add(Dense(n_classes, activation='softmax'))
        self.model.compile(loss=self.loss, optimizer=self.optimizer, metrics=self.metrics)

        self.name = f"{self.tag}_{self.loss_name}_batch{self.batch_size}_{self.optimizer}"
        self.history = None
        self.val_macroF1 = None
        self.eval_df = None
    
    def describe(self):
        return f"loss={self.loss_name}, batch_size={self.batch_size}, optimizer={self.optimizer}, explicit-class-weights: {type(self.class_weight)==np.ndarray}, embedd-trainable: {self.model.layers[0].trainable}"
    
    def summary(self):
        print(self.describe())
        return self.model.summary()
        
    def save_embedd(self):
        file = os.path.join(working_dir, f"{self.name}_weights.p")
        with open(file, 'wb') as f:
            pickle.dump(self.model.layers[0].get_weights(), f)

    def load_embedd(self, embedd_file, trainable=False):
        with open(embedd_file, "rb") as f:
            weights = pickle.load(f)
            self.model.layers[0].set_weights(weights)
            self.model.layers[0].trainable = trainable
            
        self.model.compile(loss=self.loss, optimizer=self.optimizer, metrics=self.metrics)

    
    @staticmethod
    def _join_hist(hist_1, hist_2):
        assert hist_1.keys() == hist_2.keys()
        for key in hist_2:
            hist_1[key] += hist_2[key]
    
    def fit(self, X_train, y_train, verbose=1, validate_on=None):        
        print(self.describe())
              
        new_history = self.model.fit(
            X_train, y_train,
            class_weight = self.class_weight,
            epochs=self.epochs,
            batch_size = self.batch_size,
            callbacks = self.callbacks,
            validation_data = validate_on,
            verbose = verbose        
            ).history
        if self.history:
            self._join_hist(self.history, new_history)
        else:
            self.history = new_history
        
        try:
            self.val_macroF1 = self.history['val_macroF1'][-1]
            print(f"Last val_macroF1: {self.val_macroF1}")
        except:
            pass

    
    def save_hist(self):
        file = os.path.join(working_dir, f"{self.name}_history.p")
        
        # numpy.float32-like values are not jsonifiable
        for function in self.history.keys():
            self.history[function] = [float(score) for score in model1.history[function]]
        
        with open(file, 'w') as f:
            json.dump(self.history, f)
    
    def load_hist(self):
        if self.history:
            print("Overwriting the history.")
        self.history = json.load(open(os.path.join(working_dir, f"{self.name}_history.p"), "r"))
        self.val_macroF1 = self.history['val_macroF1'][-1]

    def Ksave(self):
        file = os.path.join(working_dir, f"{self.name}_Kmodel.h5")
        save_model(self.model, file)
    
    def Kload(self):
        file = os.path.join(working_dir, f"{self.name}_Kmodel.h5")
        print("Ovewriting the model.")
        self.model = load_model(file, custom_objects=CUSTOM_OBJECTS)
        
    def discard(self):
        del self.history
        self.history = None
        del self.model
        self.model = None
    
    def plot(self, with_loss=False, with_lr=False):
        print(self.describe())
        plot_history(self.history, with_loss=with_loss, with_lr=with_lr)

    def evaluate(self, X_test, y_test):
        predict = {}
        
        # The model.evaluate method gives strangely bad results.
        # I must be misunderstanding something.
        # I've added evaluation using model.predict
        
        y_pred = self.model.predict(X_test)
        
        Ky_test = K.variable(y_test)
        Ky_pred = K.variable(y_pred)
        
        # categorical crossentropy (custom loss function)
        predict['cat_cross'] = K.eval(cat_cross(Ky_test, Ky_pred))
        # my_loss (custom loss function)
        predict['my_loss'] = K.eval(my_loss(Ky_test, Ky_pred))
        # categorical accuracy (custom metric)
        predict['cat_acc'] = K.eval(cat_acc(Ky_test, Ky_pred))
        # Macro precision (custom metric)
        predict['macroPrec'] = K.eval(macroPrec(Ky_test, Ky_pred))
        # Macro F1 (custom keras metric)
        predict['macroF1'] = K.eval(macroF1(Ky_test, Ky_pred))
#         # should match the above:
#         # Macro F1 (computed by sklearn)
#         predict['macroF1_sklearn'] = f1_score(np.round(y_pred).astype('int'), y_test.astype('int'), average='macro')
        # Macro recall (custom metric)
        predict['macroRecall'] = K.eval(macroRecall(Ky_test, Ky_pred))
        
        predict_df = pd.DataFrame(predict, index=[self.tag]).T
        
#         # the keras' build-in evaluate:
#         print(f'Loss function: {self.loss_name}. Metrics: {[metric.__name__ if callable(metric) else metric for metric in self.metrics]}')
#         columns = [self.loss_name] + [metric.__name__ if callable(metric) else metric for metric in self.metrics]
#         evaluate_results = self.model.evaluate(X_test, y_test, verbose=1)
#         evaluate_results_df = pd.DataFrame(np.asarray(evaluate_results).reshape(1,-1), columns=columns)
        
#         eval_df = pd.concat([predict_df, evaluate_results_df]).T      

        
        self.eval_df = predict_df
        
        return self.eval_df